<a href="https://colab.research.google.com/github/jjandnn/ReenactGAN/blob/master/ReenactGAN_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Citation

inproceedings{wayne2018reenactgan,

 author = {Wu, Wayne and Zhang, Yunxuan and Li, Cheng and Qian, Chen and Loy, Chen Change},

 title = {ReenactGAN: Learning to Reenact Faces via Boundary Transfer},
 booktitle = {ECCV},

 month = September,

 year = {2018}
}

In [0]:
# 查看设备，确认GPU情况
! /opt/bin/nvidia-smi

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
# 将自己之前保存在云盘中的训练模型复制到colab的用户目录里来，如果是初次运行，跳过这一步，云盘的目录视自己的目录结构而定
!cp -r '/content/drive/My Drive/Colab Notebooks/github_projects/ReenactGAN-checkpoints/checkpoints' '/content/checkpoints'

In [0]:
# 下载软件和数据集
!rm -rf sample_data
!git clone https://github.com/wywu/ReenactGAN.git
!mv ReenactGAN/* .
!rm -rf ReenactGAN
!pip install dominate
!gdown https://drive.google.com/uc?id=1jQ6d76T5GQuvQH4dq8_Wq1T0cxvN0_xp

# 解压数据集这步在云端挺耗时，要有耐心
!tar xvf CelebV.tar.gz
!rm -rf CelebV.tar.gz

## 训练模型

In [0]:
# 训练边界转换器(指的是脸部的landmark图转换)
# 数据集：CelebV
# --root_dir表示数据集的目录，--which_target的target表示要作为目标的人
# --which_target可选参数范本： 0：伊曼纽尔·马克龙 1：凯瑟琳 2：马云 3：特蕾莎·梅 4：唐纳德·特朗普，sh中默认选的是4，即川普
# --gpu_ids 0 由于colab是k80或p100就一个GPU，所以就默认设备 0 。自己跑本地，有几个排序写几个，如：--gpu_ids 0,1,2,3 代表4个GPU
# 其他参数打开train_Transformer.sh，自行查看

!bash scripts/train_Transformer_colab.sh

------------ Options -------------
batchSize: 4
beta1: 0.5
beta2: 0.999
bound_size: 64
checkpoints_dir: ./checkpoints/
component: Transformer
continue_train: False
dataset_mode: transformer
default_r: 3
display_freq: 2000
display_winsize: 256
drop_last: True
fineSize_F1: 256
gpu_ids: [0]
init_type: transformer
input_nc: 15
isTrain: True
lam_align: 10.0
lam_pix: 50.0
load_path: 
lr: 5e-05
max_dataset_size: inf
max_step: 500002
mirror: True
model_dir: ./checkpoints/Transformer_2019-11-10_13-41-20
model_name: Transformer_2019-11-10_13-41-20
nThreads: 12
n_layers_D: 3
name: Transformer_2019-11-10_13-41-20
name_landmarks_list: all_98pt.txt
ndf: 64
ngf: 64
no_dropout: True
no_html: False
no_lsgan: False
norm: batch
normalisation_type: regular
normalise: True
output_nc: 15
pca_dim: 3
pretrain_root: ./pretrained_models
print_freq: 100
random_seed: 123
root_dir: ['./CelebV/']
rotate_range: 5
save_epoch_freq: 2000
serial_batches: False
translate_range: 10
update_html_freq: 2000
weight_decay: 0.0

In [32]:
# 训练解码器
# 数据集：CelebV
# --root_dir在这里代表目标人物的目录
# 其他参数自行查看train_Decoder.sh

!bash scripts/train_Decoder.sh

------------ Options -------------
alpha: 0
batchSize: 16
beta1: 0.5
checkpoints_dir: ./checkpoints
component: Decoder
continue_train: False
crop_type: random_crop
dataset_mode: aligned_face2boundary2face
display_freq: 100
display_single_pane_ncols: 0
display_winsize: 256
drop_last: False
epoch_count: 1
feature_loss: True
feature_loss_type: relu2_2_and_relu3_3
fineSize_Boundary: 64
fineSize_F1: 256
fineSize_F2: 256
gpu_ids: [0]
identity: 0.5
init_type: normal
input_nc: 3
input_type_D: type_2
isTrain: True
lambda_A: 10.0
lambda_B: 10.0
lambda_feature_loss: 10.0
lambda_pix_loss: 100.0
load_path: 
lr: 0.0002
lr_decay_iters: 50
lr_policy: lambda
max_dataset_size: inf
mirror: True
model_dir: ./checkpoints/Decoder_2019-11-10_04-23-19
model_name: Decoder_2019-11-10_04-23-19
momentum: 0
nThreads: 32
n_layers_D: 3
name: Decoder_2019-11-10_04-23-19
name_landmarks_list: all_98pt.txt
nc_Boundary: 15
nc_F1: 3
nc_F2: 3
ndf: 64
ngf: 64
niter: 100
niter_decay: 50
no_dropout: False
no_flip: False
no_ht

KeyboardInterrupt: ignored

In [0]:
# 保存训练模型的进度到谷歌云盘，请根据自己的目录保存
!cp -r '/content/checkpoints' '/content/drive/My Drive/Colab Notebooks/github_projects/ReenactGAN-checkpoints/'

In [0]:
# 复制自己训练的模型到预训练文件夹，日期根据具体模型的时间节点来填写，人物也是
!bash /content/scripts/move_models.sh ./checkpoints/Transformer_2019-xx-xx_xx-xx-xx/G_BA_xx.pth ./checkpoints/Decoder_2019-xx-xx_xx-xx-xx/xx_net_G.pth trump
# 运行测试，生成图片
!bash /content/scripts/test.sh

## 不训练，直接测试软件自带的预训练模型生成图片，查看效果

In [0]:
# test
# 下载单人的边界转换器和编码器预训练模型：川普
!gdown https://drive.google.com/uc?id=1MBWABJK9webZxAMvN9Cl5FBhXateppzu
!gdown https://drive.google.com/uc?id=1v-8kh0N56alKiSoBAENXp9KNJ0lg_Qtq
!mv trump_net_decoder.pth pretrained_models/trumpcelebv_net_decoder.pth
!mv trump_net_transformer.pth pretrained_models/trumptmp2_net_transformer.pth

In [0]:
# 运行测试，生成图片
!bash /content/scripts/test.sh